# Composite types 

Julia allows us to define new types; in fact, many Julia types, such as `Rational` and `Complex`, are defined directly in Julia code, and are on the same footing as the types we can define ourselves -- just try `methods(Rational)`, for example, to look at the code.

In [23]:
methods(Rational)

# 4 methods for generic function "(::Type)":
(::Type{Rational})(n::Integer) at rational.jl:15
(::Type{Rational}){T<:Integer}(n::T, d::T) at rational.jl:13
(::Type{Rational})(n::Integer, d::Integer) at rational.jl:14
(::Type{T}){T}(arg) at sysimg.jl:53

Suppose we wish to define a type to represent a rational number. We would need to store the numerator and denominator, and define functions such as `+` acting on two rationals, or on a rational and an integer, etc.

Since we now already know how to extend arithmetic operators and other functions to act on rationals, we only need to know the syntax for declaring a new so-called **composite type** (often called a structure, record, or class in other languages). It looks like the following:

In [2]:
type MyRational
    numerator :: Int
    denominator :: Int
end

This declares what an object (basically, a black box with contents) of type `MyRational` looks like, i.e. which variables it contains, and what types those variables have. 

In fact, it also (by default) declares a *constructor*, a function with the same name as the type that is used to create objects of the type:

[1] Use `methods` to find out which versions of the constructor have been created automatically for our new type. Create some objects. 

In [3]:
methods(MyRational)

# 3 methods for generic function "(::Type)":
MyRational(numerator::Int64, denominator::Int64) at In[2]:2
MyRational(numerator, denominator) at In[2]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [4]:
r1 = MyRational(3,4)

MyRational(3,4)

An object `r` has two fields which are accessed by `r.numerator` and `r.denominator`. You can see the list of available properties of an object using tab completion or the `names` function [changed to `fieldnames` in v0.4]. 

In [8]:
println(r1.numerator)
println(r1.denominator)

3
4


[2] Define `*` and `+` for two objects of the `MyRational` type. [Hint: What should these functions return?]

In [ ]:
import Base.+
function +(a::MyRational, b::MyRational)
    numerator = a.numerator*b.denominator + b.numerator*a.denominator
    denominator = a.denominator*b.denominator
    cd = gcd(numerator, denominator)
    numerator = numerator/cd
    denominator = denominator/cd
    return MyRational(numerator, denominator)
end
    

In [26]:
import Base.*
function *(a::MyRational, b::MyRational)
    numerator = a.numerator*b.numerator
    denominator = a.denominator*b.denominator
    cd = gcd(numerator, denominator)
    numerator = numerator/cd
    denominator = denominator/cd
    return MyRational(numerator, denominator)
end

* (generic function with 151 methods)

The definition of the built-in `Rational` type takes a very similar form.

As with any generic function, we can ourselves add methods:

[3] Add a method to the constructor that takes a single integer and constructs the corresponding rational number.
Such methods are called *outer constructors*, since they are placed outside the definition of the type.

In [37]:
MyRational(a::Integer) = MyRational(a, 1)

MyRational

In [41]:
import Base.-
function -(a::MyRational, b::MyRational)
    return a + MyRational(-1)*b
end

- (generic function with 191 methods)

In [42]:
a = MyRational(12,16)
b = MyRational(2, 3)

println(a + b)
println(a * b)
println(a - b)

MyRational(17,12)
MyRational(1,2)
MyRational(1,12)


## Exercise: `Vector2D`

As we have seen, Julia's arrays act both as storage and as vectors. Currently, there are no *fixed-size* vectors in base Julia (although this is expected to change in the future). Nonetheless, fixed size arrays are important in many aplications. [There are the [`ImmutableArrays`](https://github.com/JuliaGeometry/ImmutableArrays.jl) and, more recently, [`FixedSizeArrays.jl`](https://github.com/SimonDanisch/FixedSizeArrays.jl) packages.] 

For example, suppose we wish to model a particle moving in 2 dimensions, with a 2-dimensional position vector and a 2-dimensional velocity vector. We could use a standard Julia array for the position and velocity, but this will, in principle, be less efficient than if we could directly use an array of fixed size 2. We can easily define such an object by hand:

[1] Define a `Vector2D` object that has `x` and `y` components that are `Float64`s.

In [3]:
type Vector2D
    x::Float64
    y::Float64
end

[2] Define `+`, multiplication by a scalar, and dot product. [For non-arithmetic operations, you must explicitly tell Julia that you are not creating a new function but are extending the previous pre-existing function. This may be done by defining explicitly `Base.dot`, or by first importing the function using `import Base: dot`.]

In [4]:
import Base.+
function +(v::Vector2D, w::Vector2D)
    sumx = v.x + w.x
    sumy = v.y + w.y
    return Vector2D(sumx, sumy)
end

+ (generic function with 164 methods)

In [5]:
import Base.*
function *(c::Float64, v::Vector2D)
    wx = c*v.x
    wy = c*v.y
    return Vector2D(wx, wy)
end

* (generic function with 151 methods)

In [6]:
function dotprod(v::Vector2D, w::Vector2D)
    return v.x*w.x + v.y*w.y
end

dotprod (generic function with 1 method)

In [7]:
v = Vector2D(3, 4)
w = Vector2D(-1, 2)

println(v + w)
println(5.0v)
println(dotprod(v, w))

Vector2D(2.0,6.0)
Vector2D(15.0,20.0)
5.0


[3] Test the relative efficiency of standard Julia vectors and these `Vector2D` objects. To do so, use the `@time` macro, wrapped around `begin...end` blocks or function calls. Don't forget to run the code once before using `@time` in order to compile the function.

In [64]:
v1 = [2, 3]
v2 = [-1, 5];

In [88]:
@time for k in 1:100000 v1 + v2 end

  0.008788 seconds (200.00 k allocations: 10.681 MB)


In [90]:
@time for k in 1:100000 v + w end

  0.002712 seconds (100.00 k allocations: 3.052 MB)


## Exercise: Particles

[3] Define a `Particle` type that contains a position and a velocity. Define a function `move!` that takes an argument $\delta t$ and moves the particle over a time step with that time.

In [34]:
type Particle
    pos::Vector2D
    vel::Vector2D
end

In [35]:
function move!(p::Particle, δt::Float64)
    pos = p.pos + δt*p.vel
    vel = p.vel
    return Particle(pos, vel)
end

move! (generic function with 2 methods)

In [36]:
p1 = Particle(Vector2D(0,0), Vector2D(1,-3))

Particle(Vector2D(0.0,0.0),Vector2D(1.0,-3.0))

In [37]:
move!(p1, 2.0)

Particle(Vector2D(2.0,-6.0),Vector2D(1.0,-3.0))

[4] Define a `Gas` type that consists of `N` particles. Write a constructor that accepts a number `N` and creates `N` particles with random positions and velocities. Define a function `move!` that moves the whole gas.

In [28]:
type Gas
    p::Particle
    
    function Gas(N)
        for count in 1:N
            x = rand()
            y = rand()
            vx = rand()
            vy = rand()
            
            pos = Vector2D(x,y)
            vel = Vector2D(vx,vy)
            
            new(p(pos,vel))
        end
    end
end

## Exercise: Automatic differentiation 

A rather interesting, but surprisingly little-known algorithmic technique is that of *automatic* (or *algorithmic*) differentiation, which allows us to differentiate a function exactly by defining a suitable type and using operator overloading.

The idea is to calculate the derivative of a complicated function by splitting it up into simple pieces. In the simplest version, this is done automatically just by defining suitable methods. Pieces may look, for example, like $u+v$, whose derivative at a point $a$ is $u'(a) + v'(a)$, or $u \cdot v$, whose derivative at $a$ is $u'(a) v(a) + u(a) v'(a)$. Thus the information required for a function $u$ is just $u(a)$ and $u'(a)$, which we will denote by $u$ and $u'$.

[1] Define a type `AutoDiff` that contains `Float64`s `u` and `u'`. [The latter may be written `u\prime<TAB>`.]

In [45]:
type AutoDiff
    u::Float64
    u′::Float64
end

[2] Using the standard rules of differentiation, define `+` and `*` such that the result gives the value and derivative of the corresponding function.

In [46]:
function +{AutoDiff}(f::AutoDiff, g::AutoDiff)
    return f.u′ + g.u′
end

function *{AutoDiff}(f::AutoDiff, g::AutoDiff)
    return f.u * g.u′ + f.u′ * g.u
end

* (generic function with 152 methods)

In [52]:
# u(x) = x^2 + 3x + 1
# u'(x) = 2x + 3

# u(3) = 19
# u'(3) = 9

# v(x) = 4 - x^3
# v'(x) = -3x^2

# v(3) = -23
# v'(3) = -27

u = AutoDiff(19,9)
v = AutoDiff(23,-27)

println(u+v)
println(u*v)
println(u*u + v*v)

-18.0
-306.0
-900.0


[3] Write a function `sin` acting on an `AutoDiff` object. Define a function $f(x) = \sin(x^2)$. You should be able to *automatically differentiate* the function if you pass it the correct type of object. What object should you pass it to calculate $f'(a)$ at a point $a$?